In [1]:
#import pytensor
#print(pytensor.config.cxx)

#set up g++ and openBLAS

#import pytensor
#print(dir(pytensor.config))

#import pytensor
#pytensor.config.blas__ldflags = '-LC:\\OpenBLAS\\lib -lopenblas'
#print(pytensor.config.blas__ldflags)

#import pytensor
#print("BLAS flags:", pytensor.config.blas__ldflags)
# print("Computation Mode:", pytensor.config.mode)


In [61]:
np.__version__

'1.26.4'

In [2]:
# pip install openpyxl

# USE ADVI (only 1000 rows for testing):

In [ ]:
import numpy as np
import pandas as pd
import pymc as pm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, precision_score, recall_score



In [1]:
# Load dataset
df = pd.read_excel("book_ratings_cleaned.xlsx")

# Select relevant columns
df = df[['User-ID', 'ISBN', 'Book-Rating']]

NameError: name 'pd' is not defined

In [ ]:
# Drop zero values as they are not the actual rating
df = df[df['Book-Rating'] > 0]

In [ ]:
# Check the size of the data
len(df)

383840

In [ ]:
df.head()

,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276744,038550120X,7
13,276747,0060517794,9


In [ ]:
# **Downsample to 1000 random rows for testing**
# df = df.sample(n=1000, random_state=42).reset_index(drop=True)

In [ ]:
# Encode User-ID and ISBN as categorical for indexing
df['User-Index'] = df['User-ID'].astype("category").cat.codes
df['Book-Index'] = df['ISBN'].astype("category").cat.codes

# **Remap indices to contiguous range** (Fixes the IndexError)
df['User-Index'] = df['User-Index'].astype("category").cat.codes
df['Book-Index'] = df['Book-Index'].astype("category").cat.codes

In [ ]:
# Compute rating counts per user and book
user_rating_counts = df.groupby('User-Index')['Book-Rating'].count()
book_rating_counts = df.groupby('Book-Index')['Book-Rating'].count()

# Avoid division by zero
user_rating_counts[user_rating_counts == 0] = 1
book_rating_counts[book_rating_counts == 0] = 1

In [ ]:
# Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Convert to numpy arrays for modeling
train_user_ids = train_df['User-Index'].values
test_user_ids = test_df['User-Index'].values
train_book_ids = train_df['Book-Index'].values
test_book_ids = test_df['Book-Index'].values
train_ratings = train_df['Book-Rating'].values # Using raw ratings for Poisson
test_ratings = test_df['Book-Rating'].values

In [ ]:
# Get updated number of unique users and books
num_users = df['User-Index'].nunique()
num_books = df['Book-Index'].nunique()

print("Number of unique users:", num_users)
print("Number of unique books:", num_books)

Number of unique users: 838
Number of unique books: 977


In [ ]:
# Set latent dimension 
latent_dim = 5

# Bayesian Probabilistic Matrix Factorization Model with Gamma-Poisson
with pm.Model() as model:
    # Prior for global mean rating
    mu = pm.Gamma("mu", alpha=2, beta=0.5)
    
    # ----- User and book bias priors (Adjust sigma for best performance) -----
    user_bias = pm.Normal("user_bias", mu=0, sigma=0.5 / np.sqrt(user_rating_counts + 1), shape=num_users)
    book_bias = pm.Normal("book_bias", mu=0, sigma=0.5 / np.sqrt(book_rating_counts + 1), shape=num_books)

    # Hierarchical priors for latent factors (beta=1 worked best)
    sigma_u = pm.HalfCauchy("sigma_u", beta=1)
    sigma_b = pm.HalfCauchy("sigma_b", beta=1)
    
    user_factors = pm.Normal("user_factors", mu=0, sigma=sigma_u, shape=(num_users, latent_dim))
    book_factors = pm.Normal("book_factors", mu=0, sigma=sigma_b, shape=(num_books, latent_dim))

    # Expected rating using Poisson lambda
    lambda_rating = pm.math.exp(
        mu +
        user_bias[train_user_ids] +
        book_bias[train_book_ids] +
        (user_factors[train_user_ids] * book_factors[train_book_ids]).sum(axis=1)
    )

    # Poisson likelihood
    ratings_obs = pm.Poisson("ratings_obs", mu=lambda_rating, observed=train_ratings)
    
    # Use ADVI for fast variational inference instead of NUTS
    print("Running Variational Inference (ADVI)...")
    approx = pm.fit(n=50000, method="advi")
    trace = approx.sample(draws=2000)

# **Extract posterior values manually since PyMC won't sample `ratings_obs`**
with model:
    print("\nManually Generating Predictions Using Posterior Samples...")
    
    # Extract posterior values
    mu_post = trace.posterior["mu"].mean().item()
    user_bias_post = trace.posterior["user_bias"].mean(dim=("chain", "draw")).values
    book_bias_post = trace.posterior["book_bias"].mean(dim=("chain", "draw")).values
    user_factors_post = trace.posterior["user_factors"].mean(dim=("chain", "draw")).values
    book_factors_post = trace.posterior["book_factors"].mean(dim=("chain", "draw")).values

    # Compute expected ratings
    predicted_ratings = np.exp(
        mu_post + 
        user_bias_post[test_user_ids] + 
        book_bias_post[test_book_ids] +
        (user_factors_post[test_user_ids] * book_factors_post[test_book_ids]).sum(axis=1)
    )

    print("\nExample of Predicted Ratings (posterior predictive mean):")
    print(predicted_ratings[:5])
    
# Evaluation Metrics
mae = mean_absolute_error(test_ratings, predicted_ratings)
rmse = np.sqrt(mean_squared_error(test_ratings, predicted_ratings))

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

Running Variational Inference (ADVI)...


Output()

Finished [100%]: Average Loss = 2,768.6



Manually Generating Predictions Using Posterior Samples...

Example of Predicted Ratings (posterior predictive mean):
[0.77044243 0.66189129 1.07396004 0.99871736 1.03573795]
Mean Absolute Error (MAE): 3.2976
Root Mean Squared Error (RMSE): 4.4131


In [ ]:
# Set latent dimension 
latent_dim = 5

# Bayesian Probabilistic Matrix Factorization Model with Gamma-Poisson
with pm.Model() as model:
    # Prior for global mean rating
    mu = pm.Gamma("mu", alpha=2, beta=0.5)
    
    # User and book bias priors
    user_bias = pm.Normal("user_bias", mu=0, sigma=1 / np.sqrt(user_rating_counts + 1), shape=num_users)
    book_bias = pm.Normal("book_bias", mu=0, sigma=1 / np.sqrt(book_rating_counts + 1), shape=num_books)

    # Hierarchical priors for latent factors
    sigma_u = pm.HalfCauchy("sigma_u", beta=1)
    sigma_b = pm.HalfCauchy("sigma_b", beta=1)
    
    user_factors = pm.Normal("user_factors", mu=0, sigma=sigma_u, shape=(num_users, latent_dim))
    book_factors = pm.Normal("book_factors", mu=0, sigma=sigma_b, shape=(num_books, latent_dim))

    # Expected rating using Poisson lambda
    lambda_rating = pm.math.exp(
        mu +
        user_bias[train_user_ids] +
        book_bias[train_book_ids] +
        (user_factors[train_user_ids] * book_factors[train_book_ids]).sum(axis=1)
    )

    # Poisson likelihood
    ratings_obs = pm.Poisson("ratings_obs", mu=lambda_rating, observed=train_ratings)
    
    # Use ADVI for fast variational inference instead of NUTS
    print("Running Variational Inference (ADVI)...")
    approx = pm.fit(n=50000, method="advi")
    trace = approx.sample(draws=2000)

# **Extract posterior values manually since PyMC won't sample `ratings_obs`**
with model:
    print("\nManually Generating Predictions Using Posterior Samples...")
    
    # Extract posterior values
    mu_post = trace.posterior["mu"].mean().item()
    user_bias_post = trace.posterior["user_bias"].mean(dim=("chain", "draw")).values
    book_bias_post = trace.posterior["book_bias"].mean(dim=("chain", "draw")).values
    user_factors_post = trace.posterior["user_factors"].mean(dim=("chain", "draw")).values
    book_factors_post = trace.posterior["book_factors"].mean(dim=("chain", "draw")).values

    # Compute expected ratings
    predicted_ratings = np.exp(
        mu_post + 
        user_bias_post[test_user_ids] + 
        book_bias_post[test_book_ids] +
        (user_factors_post[test_user_ids] * book_factors_post[test_book_ids]).sum(axis=1)
    )

    print("\nExample of Predicted Ratings (posterior predictive mean):")
    print(predicted_ratings[:5])

Running Variational Inference (ADVI)...


Output()

Finished [100%]: Average Loss = 2,790.4


KeyboardInterrupt: 

In [ ]:
# Evaluation Metrics
mae = mean_absolute_error(test_ratings, predicted_ratings)
rmse = np.sqrt(mean_squared_error(test_ratings, predicted_ratings))

print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")

Mean Absolute Error (MAE): 3.2976
Root Mean Squared Error (RMSE): 4.4131


In [ ]:
# Evaluation of Precision, Recall, MAE, and RMSE

def evaluate_predictions(true_ratings, predicted_ratings, threshold=7):
    mae = mean_absolute_error(true_ratings, predicted_ratings)
    rmse = np.sqrt(mean_squared_error(true_ratings, predicted_ratings))
    
    # Convert to binary relevance (1 if rating >= threshold, else 0)
    true_binary = (true_ratings >= threshold).astype(int)
    predicted_binary = (predicted_ratings >= threshold).astype(int)
    
    precision = precision_score(true_binary, predicted_binary, average='micro')
    recall = recall_score(true_binary, predicted_binary, average='micro')
    
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")

# Running evaluation
predicted_train_ratings = np.exp(
    trace.posterior["mu"].mean().item() +
    trace.posterior["user_bias"].mean(dim=("chain", "draw")).values[train_user_ids] +
    trace.posterior["book_bias"].mean(dim=("chain", "draw")).values[train_book_ids]
)
predicted_test_ratings = np.exp(
    trace.posterior["mu"].mean().item() +
    trace.posterior["user_bias"].mean(dim=("chain", "draw")).values[test_user_ids] +
    trace.posterior["book_bias"].mean(dim=("chain", "draw")).values[test_book_ids]
)

evaluate_predictions(train_ratings, predicted_train_ratings)
evaluate_predictions(test_ratings, predicted_test_ratings)

MAE: 1.3869
RMSE: 1.8813
Precision: 0.7037
Recall: 0.7037
MAE: 3.2975
RMSE: 4.4130
Precision: 0.6950
Recall: 0.6950


In [ ]:
# ---- Bayes General Multi-Step Lookahead Recommendation ---- #

def bayes_general_recommendation(user_index, book_indices, trace, top_k=5, exploration_factor=0.5, regret_threshold=0.8, max_regret=2.0):
    """
    Multi-step lookahead Bayesian regret minimization for recommending 5 books.
    """
    mu_samples = trace.posterior["mu"].values
    user_bias_samples = trace.posterior["user_bias"].values[:, :, user_index]
    book_bias_samples = trace.posterior["book_bias"].values[:, :, book_indices]
    user_factors_samples = trace.posterior["user_factors"].values[:, :, user_index, :]
    book_factors_samples = trace.posterior["book_factors"].values[:, :, book_indices, :]

    num_samples = mu_samples.shape[1]  # Number of posterior samples
    
    # Compute expected rewards using posterior sampling
    expected_rewards = np.mean(
        np.exp(mu_samples[:, :, None] + user_bias_samples[:, :, None] + book_bias_samples +
               np.sum(user_factors_samples[:, :, None, :] * book_factors_samples, axis=-1)), axis=1
    )

    # Compute variance (uncertainty measure)
    rating_uncertainty = np.var(
        np.exp(mu_samples[:, :, None] + user_bias_samples[:, :, None] + book_bias_samples +
               np.sum(user_factors_samples[:, :, None, :] * book_factors_samples, axis=-1)), axis=1
    )
    
    # Compute Bayesian regret
    best_expected_reward = np.max(expected_rewards, axis=1)
    regrets = best_expected_reward[:, None] - expected_rewards

    # Cap regret to prevent extreme exploration
    regrets = np.clip(regrets, 0, max_regret)

    # Apply regret threshold
    should_explore = regrets > regret_threshold

    # Compute future learning potential
    expected_future_gain = exploration_factor * rating_uncertainty

    # Compute exploration-adjusted score
    exploration_score = expected_rewards + expected_future_gain

    # Rank books
    ranked_books = np.argsort(-exploration_score, axis=1)  # Sort in descending order

    # Select top-k books for recommendation
    selected_books = [book_indices[i] for i in ranked_books[0, :top_k]]

    return selected_books

# Example usage: Recommend 5 books for a user
user_id_example = 42  # Replace with an actual user ID
book_pool = np.arange(num_books)  # Assuming all books are available

recommended_books = bayes_general_recommendation(user_id_example, book_pool, trace, top_k=5)
print("\nTop-5 Recommended Books for User", user_id_example, ":", recommended_books)


Top-5 Recommended Books for User 42 : [586, 438, 140, 27, 555]


In [ ]:
# Evaluation of Precision, Recall, MAE, and RMSE in the top 5 recommendations

def evaluate_recommendations(user_ids, book_pool, trace, top_k=5):
    total_precision = 0
    total_recall = 0
    total_mae = 0
    total_rmse = 0
    user_count = len(user_ids)
    
    for user in user_ids:
        actual_books = set(test_df[test_df['User-Index'] == user]['Book-Index'].values)
        actual_ratings = test_df[test_df['User-Index'] == user]['Book-Rating'].values
        recommended_books = set(bayes_general_recommendation(user, book_pool, trace, top_k))
        
        if len(actual_books) > 0:
            precision = len(recommended_books & actual_books) / top_k
            recall = len(recommended_books & actual_books) / len(actual_books)
            predicted_ratings = np.array([trace.posterior["mu"].mean().item() + trace.posterior["user_bias"].mean(dim=("chain", "draw")).values[user] + trace.posterior["book_bias"].mean(dim=("chain", "draw")).values[book] for book in actual_books])
            
            mae = mean_absolute_error(actual_ratings, predicted_ratings)
            rmse = np.sqrt(mean_squared_error(actual_ratings, predicted_ratings))
        else:
            precision = 0
            recall = 0
            mae = 0
            rmse = 0
        
        total_precision += precision
        total_recall += recall
        total_mae += mae
        total_rmse += rmse
    
    avg_precision = total_precision / user_count
    avg_recall = total_recall / user_count
    avg_mae = total_mae / user_count
    avg_rmse = total_rmse / user_count
    
    print(f"Average Precision@{top_k}: {avg_precision:.4f}")
    print(f"Average Recall@{top_k}: {avg_recall:.4f}")
    print(f"Mean Absolute Error (MAE): {avg_mae:.4f}")
    print(f"Root Mean Squared Error (RMSE): {avg_rmse:.4f}")

# Running evaluation
book_pool = np.arange(num_books)
evaluate_recommendations(test_user_ids, book_pool, trace, top_k=5)


Average Precision@5: 0.0010
Average Recall@5: 0.0050
Mean Absolute Error (MAE): 2.9559
Root Mean Squared Error (RMSE): 3.0136


## new metrics for ranking

In [ ]:
#find books that user would actually like (lets say 7 rating or above)
actual_books = {}
threshold = 7  # Define threshold for relevant books

for user in test_df['User-Index'].unique():
    actual_books[user] = set(test_df[(test_df['User-Index'] == user) & (test_df['Book-Rating'] >= threshold)]['Book-Index'].values)

#get recommended books for each user
recommended_books = {}

for user in test_df['User-Index'].unique():
    recommended_books[user] = bayes_general_recommendation(user, df['Book-Index'].unique(), trace, top_k=5)


#create book popularity variable
book_popularity = df['Book-Index'].value_counts().to_dict()

In [ ]:
# Mean Reciprocal Rank (MRR)
def mean_reciprocal_rank(recommended_books, actual_books):
    """
    Computes the Mean Reciprocal Rank (MRR).
    recommended_books: list of recommended book indices for each user.
    actual_books: list of sets containing relevant book indices for each user.
    """
    reciprocal_ranks = []
    for rec, actual in zip(recommended_books, actual_books):
        rank = next((i+1 for i, book in enumerate(rec) if book in actual), None)
        if rank:
            reciprocal_ranks.append(1 / rank)
        else:
            reciprocal_ranks.append(0)
    return np.mean(reciprocal_ranks)

# Normalized Discounted Cumulative Gain (NDCG)
def ndcg_at_k(recommended_books, actual_books, k=5):
    """
    Computes the Normalized Discounted Cumulative Gain (NDCG) at K.
    recommended_books: list of recommended book indices for each user.
    actual_books: list of sets containing relevant book indices for each user.
    """
    def dcg(recs, actual):
        return sum((1 / np.log2(i+2)) if rec in actual else 0 for i, rec in enumerate(recs[:k]))

    ndcg_scores = []
    for rec, actual in zip(recommended_books, actual_books):
        actual_relevances = [1 if book in actual else 0 for book in rec[:k]]
        ideal_dcg = dcg(sorted(actual_relevances, reverse=True), actual)
        actual_dcg = dcg(rec, actual)
        ndcg_scores.append(actual_dcg / ideal_dcg if ideal_dcg > 0 else 0)
    return np.mean(ndcg_scores)

# Coverage
def coverage(recommended_books, total_books):
    """
    Measures recommendation diversity as the percentage of books recommended.
    recommended_books: list of recommended book indices for each user.
    total_books: total number of books in the dataset.
    """
    unique_books = set(book for rec in recommended_books for book in rec)
    return len(unique_books) / total_books

# Novelty (measuring unexpectedness)
def novelty(recommended_books, book_popularity, k=5):
    """
    Computes novelty based on how rare the recommended books are.
    book_popularity: Dictionary mapping book index to its popularity score.
    """
    novelty_scores = []
    for rec in recommended_books:
        avg_popularity = np.mean([book_popularity.get(book, 0) for book in rec[:k]])
        novelty_scores.append(1 / (1 + avg_popularity))  # Lower popularity → higher novelty
    return np.mean(novelty_scores)

# Example Usage
# recommended_books = [[101, 203, 405], [312, 120, 305]]  # Example user recommendations
# actual_books = [{101, 405}, {120}]  # Example actual relevant books
# total_books = 1000  # Assume dataset has 1000 books
# book_popularity = {101: 500, 203: 100, 405: 50, 312: 300, 120: 20, 305: 80}  # Example popularity

# print("MRR:", mean_reciprocal_rank(recommended_books, actual_books))
# print("NDCG@5:", ndcg_at_k(recommended_books, actual_books, k=5))
# print("Coverage:", coverage(recommended_books, total_books))
# print("Novelty:", novelty(recommended_books, book_popularity, k=5))
